In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import scipy.stats

/om/user/ysa/fmri_env/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
os.listdir('.')

['summary_kids_tsl.csv',
 'Untitled.ipynb',
 'summary_adults_tsl.csv',
 'ASD_summary.csv',
 'children_all.csv',
 'summary_adults_vsl.csv',
 'get_asd.ipynb',
 '.ipynb_checkpoints',
 'summary_kids_vsl.csv']

In [2]:
def scat_plot(x, y, xlabel, ylabel):
    plt.figure(figsize=(10, 7))
    sns.regplot(
        x, y, 
        scatter_kws={'s':30}
    )
    plt.xlabel(xlabel, size=20); plt.ylabel(ylabel, size=20)
    plt.xticks(size=15); plt.yticks(size=15)
    return

In [12]:
'''
scat_plot(
    **{
        'x':df.lm_coef_rt, 
        'y':df.acc, 
        'xlabel':'lm Coef RT', 
        'ylabel':'Accuracy'
    }
)
''';

In [3]:
def drop_rows(data):
    c = []
    for idx in range(data.shape[0]):
        n_p = pd.isnull(data.loc[idx][8:-1]).mean()
        if n_p > .4:
            c.append(idx)
    return data.drop(c, 0)

def id_sort(data, id_sorter, sorting_col):
    tmp = data.set_index(id_sorter).loc[sorting_col]
    tmp['ID'] = tmp.index.values
    tmp.index = [x for x in range(tmp.shape[0])]
    return tmp

def reidx(data):
    copy = data.copy()
    copy.index = [x for x in range(copy.shape[0])]
    return copy

In [34]:
df=pd.read_csv('summary_kids_vsl.csv', index_col=0)
other_data = pd.read_csv('children_all.csv')
print other_data.columns.values
od = id_sort(other_data, 'ID', df.id_rt)
np.all(df.id_rt == od.ID)
drop_pars = od[od.group != 'TD'].ID.values
use = od[~od.ID.isin(drop_pars)]
df2 = df.set_index('id_rt').loc[use.ID]
df2.index=[x for x in range(df2.shape[0])]
use.index=[x for x in range(use.shape[0])]
mod_data = pd.concat([df2, use], axis=1)

['ID' 'gender' 'group' 'SLAge' 'BehAge' 'AgeInt' 'celf_rs' 'cls' 'kbit'
 'ctopp_e' 'ctopp_bw' 'ctopp_md' 'ctopp_nwr' 'ran_o' 'ran_c' 'ran_l'
 'ran_s' 'trog' 'wrmt_wi' 'wrmt_wa' 'towre_swe' 'towre_rd' 'towre_total'
 'wjiii' 'awma_x' 'awma_b' 'scq' 'srs']


In [85]:
X = mod_data[['acc','kbit','wjiii','mean_rt','lm_coef_rt','trog','ID']].dropna()
X = X - X.mean(0)
X = X * 1./X.std(0)

In [90]:
import statsmodels.formula.api as smf



mod = smf.mixedlm(
    formula = "acc ~ wjiii + trog + kbit + mean_rt", 
    data = X, 
    groups = X['ID'], 
    re_formula = "~mean_rt + kbit"
)
fit = mod.fit()
print(fit.summary())

                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     acc     
No. Observations:       30          Method:                 REML    
No. Groups:             30          Scale:                  0.0000  
Min. group size:        1           Likelihood:             -41.3943
Max. group size:        1           Converged:              Yes     
Mean group size:        1.0                                         
--------------------------------------------------------------------
                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                  0.041    0.302  0.137 0.891 -0.551  0.634
wjiii                     -0.178    0.159 -1.118 0.263 -0.489  0.134
trog                       0.021    0.232  0.089 0.929 -0.434  0.476
kbit                       0.326    0.312  1.042 0.297 -0.287  0.938
mean_rt                   -0.403    0.271 -1.484 

In [31]:
def drop_rows(data):
    c = []
    for idx in range(data.shape[0]):
        n_p = pd.isnull(data.loc[idx][8:-1]).mean()
        if n_p > .4:
            c.append(idx)
    return data.drop(c, 0)
    

In [33]:
nna = dr(c)
wjiii_lm_coef = nna[['lm_coef_rt', 'wjiii']]

In [36]:
scipy.stats.(wjiii_lm_coef.dropna().lm_coef_rt, wjiii_lm_coef.dropna().wjiii)

Ttest_relResult(statistic=-15.572376443610754, pvalue=4.4087165017187938e-18)